In [1]:
import numpy as np
from sval import sval
from global_vars import mH, q
from scipy.ndimage import shift

In [18]:
def create_shifted_maxwellian_include(vr: np.ndarray,       # (40,) - Array of radial velocity values
                                      vx: np.ndarray,       # (80,) - Array of x-velocity values
                                      Tnorm: float,         # Scalar value for temperature normalization
                                      vx_shift: np.ndarray, # (80,) - Array of x-velocity shifts
                                      Tmaxwell: np.ndarray, # (80,) - Array of Maxwellian temperature values
                                      Shifted_Maxwellian_Debug: int, # Integer flag for debugging
                                      mu: float,            # Scalar value for mu (2.0 in this case)
                                      mol: int,             # Integer value for molecule type
                                      nx: int,              # Integer value for the mesh size in x
                                      nvx: int,             # Integer value for the mesh size in vx
                                      nvr: int,             # Integer value for the mesh size in vr
                                      vth: float,           # Scalar value for thermal velocity
                                      vth2: float,          # Scalar value for squared thermal velocity
                                      maxwell: np.ndarray,  # (80, 80, 40) - 3D array of Maxwellian distribution function
                                      vr2vx2_ran2: np.ndarray, # (80, 40) - 2D array of vr2vx2 relation
                                      Vr2pidVr: np.ndarray, # (40,) - Array of Vr2pidVr values
                                      dVx: np.ndarray,      # (80,) - Array of differences in vx
                                      Vol: np.ndarray,      # (80, 40) - 2D array of volumes
                                      Vth_DVx: np.ndarray,  # (82, 42) - 2D array for Vth_DVx relation
                                      Vx_DVx: np.ndarray,   # (82, 42) - 2D array for Vx_DVx relation
                                      Vr_DVr: np.ndarray,   # (82, 42) - 2D array for Vr_DVr relation
                                      vr2vx2_2D: np.ndarray, # (80, 40) - 2D array of vr2vx2 in 2D
                                      jpa: int,             # Integer value for index jpa
                                      jpb: int,             # Integer value for index jpb
                                      jna: int,             # Integer value for index jna
                                      jnb: int) -> np.ndarray: # Integer value for index jnb; output is maxwell
    # Inicializar arrays
    AN = np.zeros((nvr, nvx, 2), dtype=np.float64)
    BN = np.zeros((nvr, nvx, 2), dtype=np.float64)
    # Definir el arreglo de signos
    sgn = np.array([1, -1], dtype=np.int32) 
    # Inicializar el array Maxwell a cero
    Maxwell = np.zeros((nvr, nvx, nx), dtype=np.float64)
    
    # Iterar sobre todos los valores de k
    for k in range(nx):
        # Si Tmaxwell[k] es mayor que 0.0, entonces hacer:
        if Tmaxwell[k] > 0.0:
            # Iterar sobre todos los valores de i
            for i in range(nvr):
                arg = -(vr[i]**2 + (vx - vx_shift[k]/vth)**2)*(mol*Tnorm/Tmaxwell[k])
                # Comparación como en IDL, los valores menores a cero, se convierten en 1, los mayores que cero, se convierten en 0
                arg = np.where(arg < 0.0, 1, 0)                
                # Calcular Maxwell: Los valores mayyores a -80, se convierten en 1, los menores en 0, y se toma el e^{1 o 0}
                Maxwell[i, :, k] = np.exp(np.where(arg > -80, 1, 0)) 

            maxwell[:, :, k] /= np.sum(Vr2pidVr * (maxwell[:, :, k] * dVx))

            if Shifted_Maxwellian_Debug:
                # Calculate vx_out1
                vx_out1 = vth * np.sum(Vr2pidVr * (maxwell[:, :, k] * (vx * dVx)))   #### Vx, vx, vX y VX son la misma variable, IDL es insensible a estos cambios de mayúsculas y minúsculas.
                
                # Compute vr2vx2_ran2
                for i in range(len(vr)):
                    vr2vx2_ran2[i, :] = vr[i]**2 + (vx - vx_out1 / vth)**2
                
                # Calculate T_out1
                weighted_maxwell = maxwell[:, :, k] * dVx
                T_out1 = (mol * mu * mH) * vth2 * np.sum(Vr2pidVr * (vr2vx2_ran2 * weighted_maxwell)) / (3 * q)
                
                # Calculate vth_local
                vth_local = 0.1 * np.sqrt((2 * Tmaxwell[k] * q)/(mol * mu * mH))
                
                # Calculate errors
                Terror = abs(Tmaxwell[k] - T_out1) / Tmaxwell[k]
                Verror = abs(vx_out1 - vx_shift[k]) / vth_local
            
            # Calcular momentos deseados
            WxD = vx_shift[k]
            ED = WxD**2 + 3 * q * Tmaxwell[k] / (mol * mu * mH)

            # Calcular momentos actuales de Maxwell
            WxMax = vth * np.sum(Vr2pidVr * (maxwell[:, :, k] * (vx * dVx)))
            EMax = vth2 * np.sum(Vr2pidVr * ((vr2vx2_2D * maxwell[:, :, k]) * dVx))
            
            # Calcular Nij y agregar ceros alrededor
            Nij = np.zeros((nvr + 2, nvx + 2))  # Crear un array de ceros con dimensiones aumentadas
            Nij[1:nvr+1, 1:nvx+1] = maxwell[:, :, k] * Vol  # Llenar el array interior con los valores de Maxwell * vol
  
            # Calcular variaciones de Nij
            Nijp1_vx_Dvx = shift(Nij * Vx_DVx, shift=( 0,-1), mode='wrap')  # Desplazamiento a la izquierda en la segunda dimensión
            Nij_vx_Dvx = Nij * Vx_DVx
            Nijm1_vx_Dvx = shift(Nij * Vx_DVx, shift=( 0, 1), mode='wrap')  # Desplazamiento a la derecha en la segunda dimensión

            Nip1j_vr_Dvr = shift(Nij * Vr_DVr, shift=(-1, 0), mode='wrap')  # Desplazamiento hacia arriba en la primera dimensión
            Nij_vr_Dvr = Nij * Vr_DVr
            Nim1j_vr_Dvr = shift(Nij * Vr_DVr, shift=( 1, 0), mode='wrap')  # Desplazamiento hacia abajo en la primera dimensión

            _AN =  shift(Nij * Vth_DVx, shift=(0, 1),mode='wrap') - (Nij * Vth_DVx)
            AN[:,:,0] = _AN[1:nvr+1,1:nvx+1]

            _AN = -shift(Nij * Vth_DVx, shift=(0,-1),mode='wrap') + (Nij * Vth_DVx)
            AN[:,:,1] = _AN[1:nvr+1,1:nvx+1]


            BN[:, jpa+1:jpb+1   , 0] =  Nijm1_vx_Dvx[   1:nvr+1 , jpa+2:jpb+2   ] - Nij_vx_Dvx[1:nvr+1  , jpa+2:jpb+2]
            BN[:,   jpa         , 0] = -Nij_vx_Dvx[     1:nvr+1 , jpa+1         ]
            BN[:,   jnb         , 0] =  Nij_vx_Dvx[     1:nvr+1 , jnb+1         ]
            BN[:,   jna:jnb     , 0] = -Nijp1_vx_Dvx[   1:nvr+1 , jna+1:jnb+1   ] + Nij_vx_Dvx[1:nvr+1  , jna+1:jnb+1]
            BN[:,      :        , 0] += Nim1j_vr_Dvr[   1:nvr+1 ,     1:nvx+1   ] - Nij_vr_Dvr[1:nvr+1  ,     1:nvx+1]

            BN[:, jpa+1:jpb+1   , 1] = -Nijp1_vx_Dvx[1:nvr+1, jpa+2:jpb+2] + Nij_vx_Dvx[1:nvr+1,    jpa+2:jpb+2 ]
            BN[:,   jpa         , 1] = -Nijp1_vx_Dvx[1:nvr+1, jpa+1      ]
            BN[:,   jnb         , 1] =  Nijm1_vx_Dvx[1:nvr+1, jnb+1      ]
            BN[:,   jna:jnb     , 1] =  Nijm1_vx_Dvx[1:nvr+1, jna+1:jnb+1] - Nij_vx_Dvx[1:nvr+1,    jna+1:jnb+1 ]
            BN[1:nvr,  :        , 1] -= Nip1j_vr_Dvr[2:nvr+1,     1:nvx+1] - Nij_vr_Dvr[1:nvr+1,        1:nvx+1 ]
            BN[0,      :        , 1] -= Nip1j_vr_Dvr[1      ,     1:nvx+1]


            Nij = Nij[1:nvr+1, 1:nvx+1]

            # Inicializar los arrays de tipo flotante
            TB1 = np.zeros(2, dtype=np.float32)
            TB2 = np.zeros(2, dtype=np.float32)

            # Inicializar la variable
            ia = 0


            while ia < 2:
                TA1 = vth * np.sum(AN[:, :, ia] * vx)
                TA2 = vth2 * np.sum(vr2vx2_2D * AN[:, :, ia])
                ib = 0

                while ib < 2:
                    if TB1[ib] == 0:
                        TB1[ib] = vth * np.sum(BN[:, :, ib] * vx)
                    if TB2[ib] == 0:
                        TB2[ib] = vth2 * np.sum(vr2vx2_2D * BN[:, :, ib])
                    
                    denom = TA2 * TB1[ib] - TA1 * TB2[ib]
                    b_Max = 0.0
                    a_Max = 0.0
                    
                    if denom != 0.0 and TA1 != 0.0:
                        b_Max = (TA2 * (WxD - WxMax) - TA1 * (ED - EMax)) / denom
                        a_Max = (WxD - WxMax - TB1[ib] * b_Max) / TA1
                    
                    if a_Max * sgn[ia] > 0.0 and b_Max * sgn[ib] > 0.0:
                        Maxwell[:, :, k] = (Nij + AN[:, :, ia] * a_Max + BN[:, :, ib] * b_Max) / Vol
                        ia = 2
                        ib = 2
                    ib += 1
                ia += 1
            Maxwell[:, :, k] = Maxwell[:, :, k] / (np.sum(Vr2pidVr * (Maxwell[:, :, k] * dVx)))

            if Shifted_Maxwellian_Debug:
                # Calcular vx_out2
                vx_out2 = vth * np.sum(Vr2pidVr * (Maxwell[:, :, k] * (Vx * dVx)))
                
                # Calcular vr2vx2_ran2
                vr2vx2_ran2 = np.zeros((nvr, len(vx)))
                for i in range(nvr):
                    vr2vx2_ran2[i, :] = vr[i]**2 + (vx - vx_out2 / vth)**2
                
                # Calcular T_out2
                T_out2 = (mol * mu * mH) * vth2 * np.sum(Vr2pidVr * ((vr2vx2_ran2 * Maxwell[:, :, k]) * dVx)) / (3 * q)
                
                # Calcular errores
                Terror2 = np.abs(Tmaxwell[k] - T_out2) / Tmaxwell[k]
                Verror2 = np.abs(vx_shift[k] - vx_out2) / vth_local
                
                # Imprimir resultados
                print(f'CREATE_SHIFTED_MAXWELLIAN=> Terror: {Terror:.4f} -> {Terror2:.4f}  Verror: {Verror:.4f} -> {Verror2:.4f}')
    return maxwell


In [43]:
import numpy as np

# Definir las matrices
array1 = np.array([[1, 2, 1],
                   [2, -1, 2]])

array2 = np.array([[1, 3],
                   [0, 1],
                   [1, 1]])

# Usar np.tensordot para replicar el operador # de IDL
result = np.tensordot(array1, array2, axes=(0, 1)).T

print(result)


[[ 7 -1  7]
 [ 2 -1  2]
 [ 3  1  3]]


In [2]:
nvr=20
nx = 145
nvx=40
AN = np.zeros((nvr, nvx, 2), dtype=np.float64)
BN = np.zeros((nvr, nvx, 2), dtype=np.float64)
sgn = np.array([1, -1], dtype=np.int32) 
maxwell = np.zeros((nvr, nvx, nx), dtype=np.float64)

In [27]:
Tmaxwell = np.array([
    1.07458243, 1.07411291, 1.07330848, 1.07237226, 1.07139482, 1.07046371,
    1.06952469, 1.06857025, 1.06762886, 1.06664294, 1.06562501, 1.06452486,
    1.06335018, 1.06206734, 1.06070972, 1.05931935, 1.05777014, 1.05612613,
    1.05442032, 1.05255237, 1.05059097, 1.04858941, 1.04645857, 1.04418413,
    1.04194452, 1.03952007, 1.03695597, 1.0343753,  1.03178785, 1.0290448,
    1.02625484, 1.02368735, 1.02149776, 1.01931148, 1.01789071, 1.02041256,
    1.0233504,  1.02576286, 1.02782064, 1.02929647, 1.02844618, 1.02140306,
    1.01990804, 1.03097577, 1.04267875, 1.0549926,  1.07574851, 1.10321339,
    1.13134163, 1.16130422, 1.21920798, 1.27812609, 1.33834572, 1.43592426,
    1.55629089, 1.67884027, 1.86839423, 2.11781743, 2.37124599, 2.8283328,
    3.34877023, 4.04774655, 5.09852383, 1.05918155, 1.0609074,  1.06264278,
    1.06439027, 1.06615266, 1.06793572, 1.0697475,  1.07159797, 1.07350328,
    1.07548384, 1.0775914,  1.07990805, 1.0825156,  1.08550447, 1.08897886,
    1.09296761, 1.09740772, 1.10227884, 1.10769784, 1.11389861, 1.12095543,
    1.12872648, 1.13707211, 1.14587135, 1.15500199, 1.16437847, 1.17398501,
    1.18380939, 1.19384194, 1.20407277, 1.21448804, 1.22506942, 1.23579754,
    1.24665448, 1.25762404, 1.26869103, 1.27984204, 1.29106715, 1.30236066,
    1.31371894, 1.32513818, 1.33661219, 1.34813479, 1.35970274, 1.37131068,
    1.38294831, 1.39460032, 1.40624668, 1.41786342, 1.42942349, 1.44089767,
    1.45225549, 1.46346521, 1.4744935,  1.48530609, 1.49586923, 1.50614732,
    1.51610841, 1.52572907, 1.53498868, 1.54386805, 1.55234912, 1.56041484,
    1.56804891, 1.57523567, 1.58195961, 1.5882041,  1.59395542, 1.59920598,
    1.60395279, 1.60819961, 1.61196075, 1.61526728, 1.6181772,  1.62079327,
    1.62329647, 1.62601365, 1.62956524, 1.6352098,  1.64570924, 1.66777864,
    1.72090028
])
vr = np.array([
    0.06271137, 0.13724259, 0.22359367, 0.32176461, 0.4317554,  0.55356604,
    0.68719655, 0.83264691, 0.98991712, 1.1590072,  1.33991712, 1.53264691,
    1.73719655, 1.95356604, 2.1817554,  2.42176461, 2.67359367, 2.93724259,
    3.21271137, 3.5
])
vx = np.array([
    -3.5, -3.21271137, -2.93724259, -2.67359367, -2.42176461, -2.1817554,
    -1.95356604, -1.73719655, -1.53264691, -1.33991712, -1.1590072,  -0.98991712,
    -0.83264691, -0.68719655, -0.55356604, -0.4317554,  -0.32176461, -0.22359367,
    -0.13724259, -0.06271137,  0.06271137,  0.13724259,  0.22359367,  0.32176461,
     0.4317554,   0.55356604,  0.68719655,  0.83264691,  0.98991712,  1.1590072,
     1.33991712,  1.53264691,  1.73719655,  1.95356604,  2.1817554,   2.42176461,
     2.67359367,  2.93724259,  3.21271137,  3.5
])
vx_shift = np.array([463.7734928, 466.607085, 472.59303086, 480.56001209, 488.13262314, 
            494.26721736, 500.77551058, 507.14977028, 513.39271452, 520.10545065, 
            527.76307883, 536.50157863, 546.47697357, 557.73584066, 570.36806887, 
            584.36816513, 599.83151224, 616.69435842, 634.99463353, 646.2323603, 
            647.3794833, 649.0131186, 651.16363573, 653.76695109, 656.80210532, 
            660.18066571, 663.61539393, 666.5500236, 668.60704839, 629.03319128, 
            559.09048088, 478.19979973, 384.38796794, 275.26380963, 183.54798032, 
            341.45629485, 515.65081141, 709.23355369, 928.36855435, 1249.52528617, 
            1646.01845052, 2146.40799416, 2676.81336237, 2745.67155669, 2813.02993998, 
            2880.0064726, 2944.96110001, 3007.68644701, 3071.03694447, 3136.70033264, 
            3201.30880008, 3268.50528092, 3340.63298751, 3419.76616149, 3504.67393648, 
            3599.19752079, 3713.88845447, 3847.21700979, 4002.14708874, 4216.21095655, 
            4472.21586454, 4815.85695149, 5288.52607125, 60.17334594, 91.60224811, 
            122.80591898, 153.75218643, 184.41799975, 214.80663295, 244.93953731, 
            274.84583889, 304.56152077, 334.1241696, 363.86022159, 394.32292971, 
            425.94297789, 459.08309161, 494.05247036, 530.20584848, 566.09226848, 
            600.89043942, 634.7610414, 668.33347981, 701.38989333, 732.8635148, 
            761.83664227, 787.57854635, 809.50990489, 827.24955691, 840.5620009, 
            849.16406501, 852.80307344, 851.28341969, 844.45559192, 832.23433665, 
            814.63969454, 791.83678289, 764.14109263, 732.01895762, 696.09524097, 
            657.12918529, 615.97984882, 573.56365403, 530.80480866, 488.58566118, 
            447.68709826, 408.74250016, 372.23753265, 338.51794354, 307.80017215, 
            280.18901691, 255.69901118, 234.27659405, 215.82098201, 200.20246036, 
            187.27850371, 176.90658147, 168.95277176, 163.29780083, 159.84354521, 
            158.51591476, 159.26778982, 162.08519124, 166.99088547, 174.04842894, 
            183.36735653, 195.10998621, 209.50022053, 226.83415434, 247.48812126, 
            271.94697226, 300.84112821, 334.97916476, 375.40278091, 423.46770218, 
            480.96535282, 550.31379091, 634.87336246, 739.50310607, 871.61951431, 
            1043.41379241, 1277.0984715, 1619.52126186, 2193.80072271, 3475.0904495])
vth = 84372.93085241246
mol = 1
Tnorm =  74.3179779562261

Vr2pidVr = np.array([0.03140146, 0.07085958, 0.13132866, 0.2123534, 0.31656726, 
                     0.44660368, 0.60509613, 0.79467805, 1.0179829, 1.27764412, 
                     1.57629518, 1.91656953, 2.30110061, 2.73252187, 3.21346679, 
                     3.74656879, 4.33446134, 4.9797779, 5.6851519, 6.31780701])
dVx = np.array([0.28728863, 0.2813787, 0.26955885, 0.25773899, 0.24591914, 
                0.23409928, 0.22227942, 0.21045957, 0.19863971, 0.18681986, 
                0.175, 0.16318014, 0.15136029, 0.13954043, 0.12772058, 
                0.11590072, 0.10408086, 0.09226101, 0.08044115, 0.09997698, 
                0.09997698, 0.08044115, 0.09226101, 0.10408086, 0.11590072, 
                0.12772058, 0.13954043, 0.15136029, 0.16318014, 0.175, 
                0.18681986, 0.19863971, 0.21045957, 0.22227942, 0.23409928, 
                0.24591914, 0.25773899, 0.26955885, 0.2813787, 0.28728863])

In [34]:
for k in range(nx):
    if Tmaxwell[k]>0.0:
        for i in range(nvr):
            arg = -(vr[i]**2 + (vx - vx_shift[k]/vth)**2)*(mol*Tnorm/Tmaxwell[k])
            arg = np.where(arg < 0.0, 1, 0)     
            maxwell[i, :, k] = np.exp(np.where(arg > -80, 1, 0))
        print(maxwell[:, :, k].shape)
        print(dVx.shape)
        np.matmul(maxwell[:,:,k],dVx)
        maxwell[:, :, k] /= np.sum(Vr2pidVr * np.sum(maxwell[:, :, k] * dVx, axis=1))
        print(maxwell) 

(20, 40)
(40,)
[[[0.00329012 0.00329012 0.00329012 ... 0.00329012 0.00329012 0.00329012]
  [0.00329012 0.00329012 0.00329012 ... 0.00329012 0.00329012 0.00329012]
  [0.00329012 0.00329012 0.00329012 ... 0.00329012 0.00329012 0.00329012]
  ...
  [0.00329012 0.00329012 0.00329012 ... 0.00329012 0.00329012 0.00329012]
  [0.00329012 0.00329012 0.00329012 ... 0.00329012 0.00329012 0.00329012]
  [0.00329012 0.00329012 0.00329012 ... 0.00329012 0.00329012 0.00329012]]

 [[0.00329012 0.00329012 0.00329012 ... 0.00329012 0.00329012 0.00329012]
  [0.00329012 0.00329012 0.00329012 ... 0.00329012 0.00329012 0.00329012]
  [0.00329012 0.00329012 0.00329012 ... 0.00329012 0.00329012 0.00329012]
  ...
  [0.00329012 0.00329012 0.00329012 ... 0.00329012 0.00329012 0.00329012]
  [0.00329012 0.00329012 0.00329012 ... 0.00329012 0.00329012 0.00329012]
  [0.00329012 0.00329012 0.00329012 ... 0.00329012 0.00329012 0.00329012]]

 [[0.00329012 0.00329012 0.00329012 ... 0.00329012 0.00329012 0.00329012]
  [0.00

In [45]:
import numpy as np

def idl_multiply(array1, array2):
    # Asegúrate de que las dimensiones sean compatibles
    if array1.shape[1] != array2.shape[0]:
        raise ValueError("El número de columnas en array1 debe ser igual al número de filas en array2")
    
    # Multiplicar las columnas de array1 por las filas de array2
    result = np.matmul(array2, array1)  # Transponemos array2 para multiplicar correctamente
    return result

# Ejemplo de uso
array1 = np.array([[1, 2, 1], [2, -1, 2]])
array2 = np.array([[1, 3], [0, 1], [1, 1]])

result = idl_multiply(array1, array2)
print(result)

[[ 7 -1  7]
 [ 2 -1  2]
 [ 3  1  3]]
